### データベースの概要把握とデータベースの操作体験用資料
- DDoS2022夏 DB / SQL講座
- 作成: Takumi Ban 2022/09/22

### 使用するライブラリの読み込み
- pandas: データ成型・加工用
- numpy: 数値計算用
- csv: csvファイルの読み込み・書き込み
- sqlite3: 軽量なRDBMS

In [84]:
import pandas as pd
import numpy as np
import csv
import sqlite3

- Google Driveをマウント

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データベース本体を作成

In [6]:
# データベースを作成(すでにある場合は接続)する関数
def create_db(dbname):
    dbname = dbname
    con = sqlite3.connect(dbname)
    con.close()
    return

In [7]:
# データベースを作成(すでにある場合は接続)
# カレントディレクトリ直下に music.db(sqliteデータベース本体) が作成される
db = 'music.db'
create_db(db)

### データベースにアクセス・データを操作するための関数
- 引数
    - `sql`(str): 実行したいSQL文を記述
    - `dbname`(str): アクセスしたいデータベース名を指定
    - `data`(tuple): INSERT文を実行するときのみ挿入したいデータを指定 それ以外の場合はNone

In [9]:
def dbaccess(sql, dbname, data):
    # DBを置く場所
    path = dbname

    # DBに接続
    con = sqlite3.connect(path)
    cur = con.cursor() # SQLが叩けるようになる

    # データの挿入
    if 'INSERT' in sql:
        cur.executemany(sql, data)
        con.commit()

    # データの変更を反映
    elif 'DELETE' in sql or 'UPDATE' in sql:
        cur.execute(sql)
        con.commit()

    # データの検索
    elif 'SELECT' in sql:
        cur.execute(sql)
        rows = cur.fetchall()
        con.close()
        return rows
    
    else:
        # SQLを実行する
        cur.execute(sql)

    # DBとの接続を解除
    con.close()

### テーブル構造
- `music_info`


|| id | artist_name | music_name | release |
| --- | --- | --- | --- | --- |
| type | INTEGER | String | String | INTEGER |

- `music_feature`

|| id | rock | pop | painful | magnificent | sad | kyomu |
| --- | --- | --- | --- | --- | --- | --- | --- |
| type | INTEGER | INTEGER | INTEGER | INTEGER | INTEGER | INTEGER | INTEGER |

## テーブルの作成
### テーブルを作成するSQL文
- 基本
    - `CREATE TABLE [tablename] ([column_name1] [data_type], [column_name2] [data_type], ... [column_name] [data_type]);`

- 例
    - `CREATE TABLE music_info(id INTEGER PRIMARY KEY, artist_name STRING, music_name STRING, release INTEGER);`
- 訳
    - `music_info`という名前のテーブルに, 整数型・主キーの`id`, 文字列型の`artist_name`, 文字列型の`music_name`, 整数型の`release`というカラムを作りなさい！

In [10]:
# 接続するデータベース
db = 'music.db'

# music_infoテーブルを作成
sql = 'CREATE TABLE music_info(id INTEGER PRIMARY KEY, artist_name STRING, music_name STRING, release INTEGER)'
dbaccess(sql, db, None)

# music_featureテーブルを作成
sql = 'CREATE TABLE music_feature(id INTEGER PRIMARY KEY, rock INTEGER, pop INTEGER, painful INTEGER, magnificent INTEGER, sad INTEGER, kyomu INTEGER)'
dbaccess(sql, db, None)

### CSVファイルをSQLiteで取り扱える形式に変換
- 引数
    - `csv_path`: 変換対象のCSVファイルのパス
- なぜCSVか
    - データの管理形式がDBシステムに近い
    - ダミーデータを作成するときなどにExcelやGoogle Spreadsheetでサクッと作成できる

In [15]:
# CSVファイルをSQLiteに挿入可能な形式に変換する関数
def create_data(csv_path):
    data_list = []
    with open(csv_path) as f:
        reader = csv.reader(f)
        for line in reader:
            data_list.append(tuple(line))
        data_list.pop(0) # ヘッダー行の削除
    return data_list

In [12]:
path1 = '/content/drive/MyDrive/Projects/2022_ddos_summer/DB_SQL/music_info.csv' #ここは自分の環境に合わせて書き換える
music_info = create_data(path1)

In [13]:
# 変換されたデータをチラ見
music_info

[('1', 'BiSH', 'プロミスザスター', '2017'),
 ('2', 'BiSH', 'オーケストラ', '2016'),
 ('3', 'BiSH', 'My Landscape', '2017'),
 ('4', 'BiSH', 'BiSH-星が瞬く夜に-', '2015'),
 ('5', 'BiSH', 'beautifulさ', '2016'),
 ('6', 'BiSH', 'PAiNT iT BLACK', '2018'),
 ('7', 'BiSH', 'DiSTANCE', '2019'),
 ('8', 'BiSH', 'NON TiE-UP', '2018'),
 ('9', 'BiSH', 'stereo future', '2018'),
 ('10', 'BiSH', 'STACKiNG', '2021'),
 ('11', 'BiSH', 'I have no idea.', '2021'),
 ('12', 'BiSH', 'CAN WE STiLL BE??', '2021'),
 ('13', 'BiSH', 'サラバかな', '2015'),
 ('14', 'BiSH', 'HiDE the BLUE', '2018'),
 ('15', 'BiSH', 'ALL YOU NEED IS LOVE', '2016'),
 ('16', 'PEDRO', '浪漫', '2020'),
 ('17', 'PEDRO', '感傷謳歌', '2020'),
 ('18', 'PEDRO', '自律神経出張中', '2018'),
 ('19', 'PEDRO', 'NIGHT NIGHT', '2019'),
 ('20', 'PEDRO', '生活革命', '2020'),
 ('21', 'PEDRO', '東京', '2021'),
 ('22', 'PEDRO', '雪の街', '2021'),
 ('23', 'PEDRO', 'いっそ僕の知らない世界の道端でのたれ死んでください', '2021'),
 ('24', 'PEDRO', 'ラブというソング', '2019'),
 ('25', 'TK from 凛として時雨', '蝶の飛ぶ水槽', '2020'),
 ('26', 'TK from 凛として時

In [14]:
path2 = '/content/drive/MyDrive/Projects/2022_ddos_summer/DB_SQL/music_feature.csv' # ここは自分の環境に合わせて書き換える
music_feature = create_data(path2)

In [16]:
# データをチラ見
music_feature

[('1', '1', '1', '0', '0', '0', '0'),
 ('2', '1', '1', '1', '1', '1', '1'),
 ('3', '1', '1', '0', '0', '0', '0'),
 ('4', '1', '0', '0', '0', '0', '0'),
 ('5', '1', '1', '0', '0', '0', '0'),
 ('6', '1', '0', '0', '0', '0', '0'),
 ('7', '1', '0', '1', '1', '1', '1'),
 ('8', '1', '0', '0', '0', '0', '0'),
 ('9', '1', '0', '1', '1', '0', '1'),
 ('10', '1', '1', '0', '1', '0', '0'),
 ('11', '1', '1', '0', '0', '0', '0'),
 ('12', '1', '1', '1', '0', '1', '0'),
 ('13', '1', '1', '1', '1', '1', '0'),
 ('14', '1', '1', '1', '1', '1', '0'),
 ('15', '1', '1', '1', '0', '0', '0'),
 ('16', '1', '0', '1', '0', '1', '1'),
 ('17', '1', '1', '0', '0', '0', '0'),
 ('18', '1', '1', '0', '0', '0', '0'),
 ('19', '1', '1', '0', '0', '0', '0'),
 ('20', '1', '0', '1', '0', '1', '0'),
 ('21', '1', '0', '0', '0', '0', '0'),
 ('22', '1', '0', '1', '1', '1', '1'),
 ('23', '1', '0', '0', '0', '0', '0'),
 ('24', '1', '1', '0', '0', '0', '0'),
 ('25', '1', '0', '1', '1', '0', '0'),
 ('26', '1', '0', '0', '1', '0', '

## データの挿入
### データを挿入するSQL文
- 基本
    - `INSERT INTO [table_name]([column_name1], [column_name2], ..., [column_name]) VALUES ([value1], [value2], ..., [value]);`
- 例
    - `INSERT INTO music_info(id, artist_name, music_name, release) VALUES(2, "amazarashi", "古いSF映画", 2011);`
- 訳
    - `music_info`テーブルの`id`, `artist_name`, `music_name`, `release`カラムそれぞれに`2`, `amazarashi`, `古いSF映画`, `2011`の値を挿入しなさい！

### SQLite独特の記法
- Pythonのタプルに成型された複数のデータを一度にINSERTする際には`VALUE`を次のように`?`で置換する
    - `INSERT INTO music_info(id, artist_name, music_name, release) VALUES(?, ?, ?, ?);`

- `music_info`テーブルにデータを挿入

In [20]:
sql = 'INSERT INTO music_info(id, artist_name, music_name, release) VALUES(?, ?, ?, ?)'
dbaccess(sql, db, music_info)

- `music_feature`テーブルにデータを挿入

In [17]:
sql = 'INSERT INTO music_feature(id, rock, pop, painful, magnificent, sad, kyomu) VALUES(?, ?, ?, ?, ?, ?, ?)'
dbaccess(sql, db, music_feature)

## テーブル内のすべてのデータの呼び出し
- 基本
    - `SELECT * FROM [table_name];`
- 例
    - `SELECT * FROM music_info;`
- 訳
    - `music_info`テーブル内のすべてのデータ(レコード)を呼び出せ！

- INSERTされたデータをDBから呼び出してチラ見

In [21]:
sql = 'SELECT * FROM music_info'
dbaccess(sql, db, None)

[(1, 'BiSH', 'プロミスザスター', 2017),
 (2, 'BiSH', 'オーケストラ', 2016),
 (3, 'BiSH', 'My Landscape', 2017),
 (4, 'BiSH', 'BiSH-星が瞬く夜に-', 2015),
 (5, 'BiSH', 'beautifulさ', 2016),
 (6, 'BiSH', 'PAiNT iT BLACK', 2018),
 (7, 'BiSH', 'DiSTANCE', 2019),
 (8, 'BiSH', 'NON TiE-UP', 2018),
 (9, 'BiSH', 'stereo future', 2018),
 (10, 'BiSH', 'STACKiNG', 2021),
 (11, 'BiSH', 'I have no idea.', 2021),
 (12, 'BiSH', 'CAN WE STiLL BE??', 2021),
 (13, 'BiSH', 'サラバかな', 2015),
 (14, 'BiSH', 'HiDE the BLUE', 2018),
 (15, 'BiSH', 'ALL YOU NEED IS LOVE', 2016),
 (16, 'PEDRO', '浪漫', 2020),
 (17, 'PEDRO', '感傷謳歌', 2020),
 (18, 'PEDRO', '自律神経出張中', 2018),
 (19, 'PEDRO', 'NIGHT NIGHT', 2019),
 (20, 'PEDRO', '生活革命', 2020),
 (21, 'PEDRO', '東京', 2021),
 (22, 'PEDRO', '雪の街', 2021),
 (23, 'PEDRO', 'いっそ僕の知らない世界の道端でのたれ死んでください', 2021),
 (24, 'PEDRO', 'ラブというソング', 2019),
 (25, 'TK from 凛として時雨', '蝶の飛ぶ水槽', 2020),
 (26, 'TK from 凛として時雨', 'yesworld', 2021),
 (27, 'TK from 凛として時雨', 'White Silence', 2012),
 (28, 'TK from 凛として時雨', 'unrave

In [ ]:
sql = 'SELECT * FROM music_feature'
dbaccess(sql, db, None)

[(1, 1, 1, 0, 0, 0, 0),
 (2, 1, 1, 1, 1, 1, 1),
 (3, 1, 1, 0, 0, 0, 0),
 (4, 1, 0, 0, 0, 0, 0),
 (5, 1, 1, 0, 0, 0, 0),
 (6, 1, 0, 0, 0, 0, 0),
 (7, 1, 0, 1, 1, 1, 1),
 (8, 1, 0, 0, 0, 0, 0),
 (9, 1, 0, 1, 1, 0, 1),
 (10, 1, 1, 0, 1, 0, 0),
 (11, 1, 1, 0, 0, 0, 0),
 (12, 1, 1, 1, 0, 1, 0),
 (13, 1, 1, 1, 1, 1, 0),
 (14, 1, 1, 1, 1, 1, 0),
 (15, 1, 1, 1, 0, 0, 0),
 (16, 1, 0, 1, 0, 1, 1),
 (17, 1, 1, 0, 0, 0, 0),
 (18, 1, 1, 0, 0, 0, 0),
 (19, 1, 1, 0, 0, 0, 0),
 (20, 1, 0, 1, 0, 1, 0),
 (21, 1, 0, 0, 0, 0, 0),
 (22, 1, 0, 1, 1, 1, 1),
 (23, 1, 0, 0, 0, 0, 0),
 (24, 1, 1, 0, 0, 0, 0),
 (25, 1, 0, 1, 1, 0, 0),
 (26, 1, 0, 0, 1, 0, 0),
 (27, 0, 1, 1, 1, 1, 1),
 (28, 1, 0, 0, 0, 1, 1),
 (29, 1, 0, 1, 1, 1, 1),
 (30, 1, 0, 0, 0, 1, 0),
 (31, 0, 1, 1, 0, 0, 1),
 (32, 1, 0, 0, 0, 0, 0),
 (33, 0, 1, 1, 1, 1, 1)]

### 呼び出したデータをJupyter Notebook上で見やすくする関数
- 引数
    - `sql`(str): 実行したいSELECT文を含むSQL

In [22]:
# 見やすくするためにデータフレーム化して表示する関数
def get_data(sql):
    con = sqlite3.connect(db)
    df  = pd.read_sql(sql, con)
    con.close()
    return df.style.hide_index()

In [23]:
sql = 'SELECT * FROM music_info;'
get_data(sql)

id,artist_name,music_name,release
1,BiSH,プロミスザスター,2017
2,BiSH,オーケストラ,2016
3,BiSH,My Landscape,2017
4,BiSH,BiSH-星が瞬く夜に-,2015
5,BiSH,beautifulさ,2016
6,BiSH,PAiNT iT BLACK,2018
7,BiSH,DiSTANCE,2019
8,BiSH,NON TiE-UP,2018
9,BiSH,stereo future,2018
10,BiSH,STACKiNG,2021


In [24]:
sql = 'SELECT * FROM music_feature;'
get_data(sql)

id,rock,pop,painful,magnificent,sad,kyomu
1,1,1,0,0,0,0
2,1,1,1,1,1,1
3,1,1,0,0,0,0
4,1,0,0,0,0,0
5,1,1,0,0,0,0
6,1,0,0,0,0,0
7,1,0,1,1,1,1
8,1,0,0,0,0,0
9,1,0,1,1,0,1
10,1,1,0,1,0,0


## SELECT文で条件に合わせたデータの検索

### 抽出するカラムの指定
- すべてのカラムを抽出するには`*`(ワイルドカード)を使用する
    - 基本
        - `SELECT * FROM [table_name];`
    - 例
        - `SELECT * FROM music_info;`
    - 訳
        - `music_info`テーブル内のすべてのカラムのすべてのデータを抽出しなさい！

- 抽出するカラムを個別に指定することもできる
    - 基本
        - `SELECT [column_name1], [column_name2] FROM [table_name];`
    - 例
        - `SELECT id, music_name FROM music_info;`
    - 訳
        - `music_info`テーブル内の`id`, `music_name`カラムのすべてのデータを抽出しなさい！

In [25]:
sql = 'SELECT id, music_name from music_info;'
get_data(sql)

id,music_name
1,プロミスザスター
2,オーケストラ
3,My Landscape
4,BiSH-星が瞬く夜に-
5,beautifulさ
6,PAiNT iT BLACK
7,DiSTANCE
8,NON TiE-UP
9,stereo future
10,STACKiNG


### WHERE句によるデータの検索
- `WHERE`句を使うと検索して抽出するデータに条件式をつけることができる
- `WHERE`句を使えるようになれば条件に合わせてデータの抽出がほぼできるようになる
- 条件式とは`[column_name]`の値がある値と同値である、ある値より大きい(小さい)など
    - 基本
        - `SELECT * FROM [table_name] WHERE [column_name]=[value];`
    - 例
        - `SELECT * FROM music_info WHERE release=2020;`
    - 訳
        - `music_info`テーブル内の`release`カラムのデータが`2020`と等しいデータをすべて抽出しなさい！

In [26]:
# releaseカラムが2020と等しい場合
sql = 'SELECT * from music_info WHERE release=2020;'
get_data(sql)

id,artist_name,music_name,release
16,PEDRO,浪漫,2020
17,PEDRO,感傷謳歌,2020
20,PEDRO,生活革命,2020
25,TK from 凛として時雨,蝶の飛ぶ水槽,2020
33,TK from 凛として時雨,copy light,2020


In [27]:
# releaseカラムが2020以上である場合 -> 2020年以降にリリースされた楽曲
sql = 'SELECT * from music_info WHERE release>=2020;'
get_data(sql)

id,artist_name,music_name,release
10,BiSH,STACKiNG,2021
11,BiSH,I have no idea.,2021
12,BiSH,CAN WE STiLL BE??,2021
16,PEDRO,浪漫,2020
17,PEDRO,感傷謳歌,2020
20,PEDRO,生活革命,2020
21,PEDRO,東京,2021
22,PEDRO,雪の街,2021
23,PEDRO,いっそ僕の知らない世界の道端でのたれ死んでください,2021
25,TK from 凛として時雨,蝶の飛ぶ水槽,2020


In [28]:
# releaseカラムが2018未満である場合 -> 2017年以前にリリースされた楽曲
sql = 'SELECT * from music_info WHERE release<2018;'
get_data(sql)

id,artist_name,music_name,release
1,BiSH,プロミスザスター,2017
2,BiSH,オーケストラ,2016
3,BiSH,My Landscape,2017
4,BiSH,BiSH-星が瞬く夜に-,2015
5,BiSH,beautifulさ,2016
13,BiSH,サラバかな,2015
15,BiSH,ALL YOU NEED IS LOVE,2016
27,TK from 凛として時雨,White Silence,2012
28,TK from 凛として時雨,unravel,2014
29,TK from 凛として時雨,invalid phrase,2016


### ORDER BY句によるデータの並び替え
- 抽出したデータを昇順・降順に並び替えたい場面は多くある
- `ORDER BY`句により条件に合わせて並び替え(ソート)ができる
- 基本
    - `SELECT [column_name] FROM [table_name] ORDER BY [column_name] [ASC or DESC];`
- 例
    - `SELECT * FROM music_info ORDER BY release ASC;`
    - `ASC`: 昇順
    - `DESC`: 降順
- 訳
    - `music_info`テーブル内のすべてのデータを`release`カラムのデータを基準に昇順に並び替えて抽出しなさい！

In [29]:
sql = 'SELECT * from music_info ORDER BY release ASC;'
get_data(sql)

id,artist_name,music_name,release
27,TK from 凛として時雨,White Silence,2012
28,TK from 凛として時雨,unravel,2014
30,TK from 凛として時雨,Dramatic Slow Motion,2014
4,BiSH,BiSH-星が瞬く夜に-,2015
13,BiSH,サラバかな,2015
2,BiSH,オーケストラ,2016
5,BiSH,beautifulさ,2016
15,BiSH,ALL YOU NEED IS LOVE,2016
29,TK from 凛として時雨,invalid phrase,2016
31,TK from 凛として時雨,like there is tomorrow,2016


In [33]:
# WHERE句と組み合わせて使うこともできる
sql = 'SELECT * from music_info WHERE artist_name="TK from 凛として時雨" ORDER BY release DESC;'
get_data(sql)

id,artist_name,music_name,release
26,TK from 凛として時雨,yesworld,2021
25,TK from 凛として時雨,蝶の飛ぶ水槽,2020
33,TK from 凛として時雨,copy light,2020
32,TK from 凛として時雨,katharsis,2018
29,TK from 凛として時雨,invalid phrase,2016
31,TK from 凛として時雨,like there is tomorrow,2016
28,TK from 凛として時雨,unravel,2014
30,TK from 凛として時雨,Dramatic Slow Motion,2014
27,TK from 凛として時雨,White Silence,2012


## テーブルの結合

### INNER JOIN句によるテーブル結合
- ある決まりに基づいて複数のテーブルを結合することができる
- `music_info`テーブル内のデータと`music_feature`テーブル内のデータは`id`によって紐付けされている
- 基本
    - ```
    SELECT
        [table_name1].[column_name1],
        [table_name2].[column_name2]
    FROM
        [table_name1]
    INNER JOIN
        [table_name2]
    ON
        [table_name1].[column_name] = [table_name2].[column_name]; -- 結合する条件
    ```
    (SQLは;(セミコロン)が打たれるまでを1つのSQL文として認識するため、複数行に渡って記述可能)
- 例
    - ```
    SELECT
        music_info.id,
        music_info.artist_name,
        music_info.music_name,
        music_info.release,
        music_feature.rock,
        music_feature.pop,
        music_feature.painful,
        music_feature.magnificent,
        music_feature.sad,
        music_feature.kyomu
    FROM
        music_info
    INNER JOIN
        music_feature
    ON
        music_info.id = music_feature.id;
    ```
- 訳
    - `music_info`テーブルと`music_feature`テーブルを`music_info`テーブルの`id`カラムと`music_feature`テーブルの`id`カラムを基準として内部結合して抽出しなさい！

In [34]:
sql = '''
SELECT
        music_info.id,
        music_info.artist_name,
        music_info.music_name,
        music_info.release,
        music_feature.rock,
        music_feature.pop,
        music_feature.painful,
        music_feature.magnificent,
        music_feature.sad,
        music_feature.kyomu
    FROM
        music_info
    INNER JOIN
        music_feature
    ON
        music_info.id = music_feature.id;
'''
get_data(sql)

id,artist_name,music_name,release,rock,pop,painful,magnificent,sad,kyomu
1,BiSH,プロミスザスター,2017,1,1,0,0,0,0
2,BiSH,オーケストラ,2016,1,1,1,1,1,1
3,BiSH,My Landscape,2017,1,1,0,0,0,0
4,BiSH,BiSH-星が瞬く夜に-,2015,1,0,0,0,0,0
5,BiSH,beautifulさ,2016,1,1,0,0,0,0
6,BiSH,PAiNT iT BLACK,2018,1,0,0,0,0,0
7,BiSH,DiSTANCE,2019,1,0,1,1,1,1
8,BiSH,NON TiE-UP,2018,1,0,0,0,0,0
9,BiSH,stereo future,2018,1,0,1,1,0,1
10,BiSH,STACKiNG,2021,1,1,0,1,0,0


In [35]:
# これにWHERE句とORDER BY句を組み合わせることも可能
sql = '''
SELECT
        music_info.id,
        music_info.artist_name,
        music_info.music_name,
        music_info.release,
        music_feature.rock,
        music_feature.pop,
        music_feature.painful,
        music_feature.magnificent,
        music_feature.sad,
        music_feature.kyomu
    FROM
        music_info
    INNER JOIN
        music_feature
    ON
        music_info.id = music_feature.id
    WHERE
        artist_name = "BiSH"
    ORDER BY
        release DESC;
'''
get_data(sql)

id,artist_name,music_name,release,rock,pop,painful,magnificent,sad,kyomu
10,BiSH,STACKiNG,2021,1,1,0,1,0,0
11,BiSH,I have no idea.,2021,1,1,0,0,0,0
12,BiSH,CAN WE STiLL BE??,2021,1,1,1,0,1,0
7,BiSH,DiSTANCE,2019,1,0,1,1,1,1
6,BiSH,PAiNT iT BLACK,2018,1,0,0,0,0,0
8,BiSH,NON TiE-UP,2018,1,0,0,0,0,0
9,BiSH,stereo future,2018,1,0,1,1,0,1
14,BiSH,HiDE the BLUE,2018,1,1,1,1,1,0
1,BiSH,プロミスザスター,2017,1,1,0,0,0,0
3,BiSH,My Landscape,2017,1,1,0,0,0,0


## せっかくなので楽曲特徴を入力するとマッチした楽曲推薦されるやつでも作ってみるか

In [114]:
# 楽曲特徴を入力させる関数
def f_input():
    print('0 or 1を入力')
    rock = int(input("rock: "))
    pop = int(input("pop: "))
    painful = int(input("painful: "))
    magnificent = int(input("magnificent: "))
    sad = int(input("sad: "))
    kyomu = int(input("kyomu: "))
    features = np.array([rock, pop, painful, magnificent, sad, kyomu])
    return features

In [137]:
# 計算して検索して表示する関数
def calc(input_features):
    # DB内の楽曲特徴をSQLで抽出
    sql = 'SELECT rock, pop, painful, magnificent, sad, kyomu FROM music_feature;'
    db_features = dbaccess(sql, db, None)

    # 内積計算
    result = np.array([])
    for dbf in db_features:
        d = np.dot(input_features, dbf)
        result = np.append(d, result)

    # 上位3曲のidを抽出
    ind = np.argsort(result)[:3]

    # 上位3曲の情報をSQLでDBから抽出
    sql = f'SELECT * FROM music_info WHERE id IN ({ind[0]}, {ind[1]}, {ind[2]});'
    return get_data(sql)

In [138]:
# 遊んでみる
calc(f_input())

0 or 1を入力
rock: 1
pop: 0
painful: 1
magnificent: 0
sad: 1
kyomu: 1


id,artist_name,music_name,release
16,PEDRO,浪漫,2020
29,TK from 凛として時雨,invalid phrase,2016
30,TK from 凛として時雨,Dramatic Slow Motion,2014


In [139]:
# もう一度遊んでみる
calc(f_input())

0 or 1を入力
rock: 0
pop: 1
painful: 0
magnificent: 1
sad: 0
kyomu: 0


id,artist_name,music_name,release
1,BiSH,プロミスザスター,2017
3,BiSH,My Landscape,2017
17,PEDRO,感傷謳歌,2020


- DB内のデータに偏り(虚無なソングばかり)が大きく影響しているようだ